In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Zuerst definieren wir Header, um unseren Bot zu kennzeichnen und rufen die Seite mit requests ab.

In [ ]:
url = 'https://de.wikipedia.org/wiki/Liste_der_Mitglieder_des_1._Ernannten_Landtages_Nordrhein-Westfalen'

headers = {
    'User-Agent': 'My User Agent',
    'From': 'kalange@statistik.tu-dortmund.de'
}
response = requests.get(url, headers=headers)
print(response)

Wir können Beautifulsoup benutzen, um die HTML zu parsen. Mit dem "find" Befehl können wir gezielt nach HTML-Elementen suchen. Hier suchen wir nach einem Element mit der ID "firstHeading", also der ersten Überschrift, und geben ihren Text aus.

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
title = soup.find('h1', id='firstHeading').get_text()
print("Titel:", title)

Genau so können wir auch nach unserer Tabelle suchen. Sie hat die Klasse "wikitable". Sobald wir sie gefunden haben, müssen wir noch den Körper der Tabelle aufrufen ("tbody") und können dann alle Zeilen ("tr") auslesen.

In [ ]:
abgeordnete = soup.find('table', class_='wikitable')
abgeordnete = abgeordnete.find("tbody")
rows = abgeordnete.find_all('tr')

Da die Tabelle etwas sperrig aufgebaut ist, müssen wir sie etwas aufbereiten. Zuerst extrahieren wir die Spaltenüberschriften aus der ersten Zeile. Dann iterieren wir über die restlichen Zeilen und fügen sie in ein Pandas DataFrame ein. Am Ende speichern wir das DataFrame als CSV-Datei.

In [ ]:
columns = rows[0].get_text().removeprefix("\n").removesuffix("\n").split("\n\n")
rows = rows[1:]
data = pd.DataFrame(columns=columns)

for row in rows:
    data = pd.concat([data,
                      pd.DataFrame([row.get_text().removeprefix("\n").removesuffix("\n").split("\n\n")],
                                   columns=columns)], ignore_index=True)

data.to_csv("abgeordneten.csv")